In [1]:
import pavey as RS
import numpy as np
import random

def build_random_sample(X,samp_axis,min_chunk=4,max_chunk=20,shuffle=True):

    nsamp = X.shape[samp_axis]
    
    idx = np.arange(nsamp)
    if shuffle:
        np.random.shuffle(idx)
    
    
        
    i0=0
    i1=np.random.randint(min_chunk,max_chunk)
    XA = []
    while True:
        XA.append(np.take(X,idx[i0:i1],axis=samp_axis))
        if i1==nsamp:
            break
        i0 = i1
        i1 += np.random.randint(min_chunk,max_chunk)
        i1 = min(nsamp,i1)
        
    return XA
    
    

In [19]:
axis = 0
X = np.random.randn(1000,4,3)
XA = build_random_sample(X,axis)


ave = np.array([x.mean(axis=axis) for x in XA])
var = np.array([x.var(axis=axis) for x in XA])
wt = np.array([x.shape[axis] for x in XA])
cov = np.array([RS.cov_nd(x,axis=axis,ddof=0) for x in XA])

(91, 4, 3, 4, 3)

In [3]:
import RunningStats as RS
reload(RS)

<module 'RunningStats' from '/Users/wpk/Documents/python/RunningStats/__init__.pyc'>

In [4]:
r = RS.RunningStatsList.from_stats(wt, ave, var=var)

In [5]:
r2 = [r.combine(x).mean_SEM() for x in range(1,20,2)]

In [9]:
np.array(r2)[:,0,0]

array([ 0.03523887,  0.03365353,  0.03149257,  0.0327831 ,  0.03522335,
        0.04427879,  0.04491904,  0.04500138,  0.03363014,  0.03794159])

In [10]:
r2.mean_SEM()

AttributeError: 'list' object has no attribute 'mean_SEM'

In [12]:
t = RS.RunningStatsVecCov(X.shape[1:])
for i in range(ave.shape[0]):
    t.push_stat(wt[i],ave[i],cov[i])

In [13]:
t2 = RS.RunningStatsVecCov.from_stats(wt, ave, cov)

In [14]:
t3 = RS.RunningStatsVecCov.from_vals(X)

In [15]:
RS.RunningStatsList.from_stats_gen(RS.RunningStatsVecCov, wt, ave, cov, axis=0)

In [16]:
tL = RS.RunningStatsList.from_stats(wt, ave, cov=cov)

In [17]:
tt = tL.combine(None)

In [18]:
tt.var() - t.var()

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])